In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


In [18]:
EPOCH = 1
BATCH_SIZE = 100
TIME_STEP= 28
INPUT_SIZE=28
LR = 0.01
log_interval = 10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    100, shuffle=True)


In [3]:

class RNNMIST(nn.Module):
    def __init__(self):
        super(RNNMIST,self).__init__()
        
        self.rnn = nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size = 64,
            num_layers = 1,
            batch_first= True
        )
        
        self.fc = nn.Linear(64,10)
        
        self.softMax = nn.Softmax()
        
    
    def forward(self,samples):
        rnn_out, (h_n,h_c ) = self.rnn(samples, None)
        tofc = rnn_out[:,-1,:]
        return self.softMax(self.fc(tofc))
  

In [37]:
#stat to train the rnn
rnn = RNNMIST()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(rnn.parameters(),lr=0.001)

def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data.view(-1,28,28)), Variable(target)
 
        output = rnn(data)
        loss = loss_func(output,target)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx * len(data) / len(train_loader.dataset), loss.data[0]))
         
def test(epoch):

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data.view(-1,28,28), volatile=True), Variable(target)
        output = rnn(data)
        test_loss += loss_func(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        


        
rnn.load_state_dict(torch.load('./rnn.mnist'))

for i in range(20):
    train(i)
    test(i)
    
    
torch.save(rnn.state_dict(),'./rnn.mnist')
print 'training end'

Train Epoch: 0 [0/60000 (0%)]	Loss: 1.492749
Train Epoch: 0 [10000/60000 (17%)]	Loss: 1.481826
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.461700
Train Epoch: 0 [30000/60000 (50%)]	Loss: 1.462090
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.480950
Train Epoch: 0 [50000/60000 (83%)]	Loss: 1.462411

Test set: Average loss: 1.4801, Accuracy: 9816/10000 (98%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.461297
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.471305
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.461469
Train Epoch: 1 [30000/60000 (50%)]	Loss: 1.471276
Train Epoch: 1 [40000/60000 (67%)]	Loss: 1.461205
Train Epoch: 1 [50000/60000 (83%)]	Loss: 1.462377

Test set: Average loss: 1.4800, Accuracy: 9819/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.461205
Train Epoch: 2 [10000/60000 (17%)]	Loss: 1.462032
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.481539
Train Epoch: 2 [30000/60000 (50%)]	Loss: 1.471456
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.461981
Train Epoch: 2 [50000/60000 (83%)]	Loss: 1.